# Fixed Adaptive Learning Physics Model - Historical Validation

**Fixed Issue**: Both models were evaluated identically despite predicting different outlet temperatures.

**Solution**: Implemented reverse prediction approach - given an achieved indoor temperature, what outlet temperature should each model have predicted?

**Key Innovation**: Compare each model's reverse-predicted outlet against actual outlet used.

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
import os
from dotenv import load_dotenv
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Import notebook helpers and enhanced model
from notebook_imports import create_influx_service
from src.thermal_equilibrium_model import ThermalEquilibriumModel

print("🔧 Fixed Adaptive Learning Physics Model - Historical Validation")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("✅ Enhanced model with REVERSE PREDICTION evaluation")

In [ ]:
# Enhanced Heat Curve with Reverse Prediction
class HeatCurveWithReverse:
    """Heat curve baseline with reverse prediction capability."""
    
    def __init__(self):
        # Your proven heat curve parameters
        self.points = {"x1": -15.0, "y1": 64.0, "x2": 18.0, "y2": 31.0}
        self.slope = (self.points["y1"] - self.points["y2"]) / (self.points["x1"] - self.points["x2"])
        self.intercept = self.points["y2"] - (self.slope * self.points["x2"])
        
        # Performance tracking for reverse predictions
        self.reverse_predictions = []
        self.reverse_errors = []
        
    def predict_outlet_temperature(self, current_indoor, target_indoor, outdoor_temp, 
                                 outdoor_forecast=None, shift_value=0):
        """Forward prediction: predict outlet temperature."""
        if outdoor_forecast is not None:
            target_temp = outdoor_temp * 0.6 + outdoor_forecast * 0.4
        else:
            target_temp = outdoor_temp
            
        outlet_temp = self.slope * target_temp + self.intercept + shift_value
        return max(16.0, min(65.0, outlet_temp))
    
    def reverse_predict_outlet_temperature(self, achieved_indoor, target_indoor, outdoor_temp,
                                         outdoor_forecast=None):
        """REVERSE prediction: Given achieved indoor temp, what outlet should we have predicted?"""
        # For heat curve, outlet depends mainly on outdoor temperature
        # But we can adjust based on how far off we were from target
        
        base_outlet = self.predict_outlet_temperature(achieved_indoor, target_indoor, outdoor_temp, outdoor_forecast)
        
        # Adjustment based on actual result vs target
        temp_error = achieved_indoor - target_indoor
        
        # If we overshot (positive error), we should have used lower outlet
        # If we undershot (negative error), we should have used higher outlet
        adjustment = -temp_error * 3.0  # 3°C outlet adjustment per 1°C indoor error
        
        reverse_predicted_outlet = base_outlet + adjustment
        return max(16.0, min(65.0, reverse_predicted_outlet))
    
    def track_reverse_prediction(self, reverse_predicted_outlet, actual_outlet, timestamp):
        """Track reverse prediction performance."""
        outlet_error = abs(reverse_predicted_outlet - actual_outlet)
        
        self.reverse_predictions.append({
            'timestamp': timestamp,
            'reverse_predicted_outlet': reverse_predicted_outlet,
            'actual_outlet': actual_outlet,
            'outlet_error': outlet_error
        })
        
        self.reverse_errors.append(outlet_error)
    
    def get_reverse_performance_metrics(self):
        """Get reverse prediction performance metrics."""
        if not self.reverse_errors:
            return {'insufficient_data': True}
            
        return {
            'avg_outlet_error': np.mean(self.reverse_errors),
            'max_outlet_error': np.max(self.reverse_errors),
            'std_outlet_error': np.std(self.reverse_errors),
            'total_predictions': len(self.reverse_predictions),
            'within_2C': sum(1 for e in self.reverse_errors if e <= 2.0) / len(self.reverse_errors) * 100,
            'within_5C': sum(1 for e in self.reverse_errors if e <= 5.0) / len(self.reverse_errors) * 100
        }

In [ ]:
# Enhanced Physics Model with Reverse Prediction
class PhysicsModelWithReverse:
    """Wrapper for ThermalEquilibriumModel with reverse prediction capability."""
    
    def __init__(self):
        self.model = ThermalEquilibriumModel()
        
        # Performance tracking for reverse predictions
        self.reverse_predictions = []
        self.reverse_errors = []
        
    def calculate_optimal_outlet_temperature(self, *args, **kwargs):
        """Forward prediction: calculate optimal outlet temperature."""
        return self.model.calculate_optimal_outlet_temperature(*args, **kwargs)
    
    def predict_equilibrium_temperature(self, *args, **kwargs):
        """Forward prediction: predict equilibrium temperature."""
        return self.model.predict_equilibrium_temperature(*args, **kwargs)
        
    def update_prediction_feedback(self, *args, **kwargs):
        """Update adaptive learning."""
        return self.model.update_prediction_feedback(*args, **kwargs)
        
    def get_adaptive_learning_metrics(self):
        """Get adaptive learning metrics."""
        return self.model.get_adaptive_learning_metrics()
        
    def reset_adaptive_learning(self):
        """Reset adaptive learning state."""
        return self.model.reset_adaptive_learning()
    
    def reverse_predict_outlet_temperature(self, achieved_indoor, target_indoor, outdoor_temp,
                                         outdoor_forecast=None, pv_power=0):
        """REVERSE prediction: Given achieved indoor temp, what outlet should we have predicted?"""
        
        # Use physics model to reverse-engineer the outlet temperature
        # We want to find outlet_temp such that our equilibrium prediction matches achieved_indoor
        
        # Binary search to find optimal outlet temperature
        low_outlet = 16.0
        high_outlet = 65.0
        tolerance = 0.1  # °C tolerance
        
        for _ in range(20):  # Maximum iterations
            mid_outlet = (low_outlet + high_outlet) / 2
            
            # Predict what indoor temp this outlet would achieve
            predicted_indoor = self.model.predict_equilibrium_temperature(
                mid_outlet, outdoor_temp, pv_power=pv_power
            )
            
            # Check if we're close enough to achieved indoor
            error = predicted_indoor - achieved_indoor
            
            if abs(error) < tolerance:
                return mid_outlet
            elif error > 0:  # Predicted too high, reduce outlet
                high_outlet = mid_outlet
            else:  # Predicted too low, increase outlet
                low_outlet = mid_outlet
                
        # Return best estimate if couldn't converge
        return (low_outlet + high_outlet) / 2
    
    def track_reverse_prediction(self, reverse_predicted_outlet, actual_outlet, timestamp):
        """Track reverse prediction performance."""
        outlet_error = abs(reverse_predicted_outlet - actual_outlet)
        
        self.reverse_predictions.append({
            'timestamp': timestamp,
            'reverse_predicted_outlet': reverse_predicted_outlet,
            'actual_outlet': actual_outlet,
            'outlet_error': outlet_error
        })
        
        self.reverse_errors.append(outlet_error)
    
    def get_reverse_performance_metrics(self):
        """Get reverse prediction performance metrics."""
        if not self.reverse_errors:
            return {'insufficient_data': True}
            
        return {
            'avg_outlet_error': np.mean(self.reverse_errors),
            'max_outlet_error': np.max(self.reverse_errors),
            'std_outlet_error': np.std(self.reverse_errors),
            'total_predictions': len(self.reverse_predictions),
            'within_2C': sum(1 for e in self.reverse_errors if e <= 2.0) / len(self.reverse_errors) * 100,
            'within_5C': sum(1 for e in self.reverse_errors if e <= 5.0) / len(self.reverse_errors) * 100
        }

In [ ]:
# Quick test to verify different predictions
heat_curve = HeatCurveWithReverse()
physics_model = PhysicsModelWithReverse()

# Test scenario
achieved_indoor = 21.2  # Actually achieved
target_indoor = 21.0    # Target was
outdoor_temp = 5.0      # Conditions
actual_outlet_used = 40.0  # What outlet was actually used

print("🧪 REVERSE PREDICTION TEST:")
print(f"Scenario: Achieved {achieved_indoor}°C (target was {target_indoor}°C)")
print(f"Outdoor: {outdoor_temp}°C, Actual outlet used: {actual_outlet_used}°C")
print()

# What should each model have predicted for outlet?
heat_curve_reverse = heat_curve.reverse_predict_outlet_temperature(achieved_indoor, target_indoor, outdoor_temp)
physics_reverse = physics_model.reverse_predict_outlet_temperature(achieved_indoor, target_indoor, outdoor_temp)

print("REVERSE PREDICTIONS (what outlet temp should each model have predicted?):")
print(f"Heat Curve reverse prediction: {heat_curve_reverse:.1f}°C")
print(f"Physics Model reverse prediction: {physics_reverse:.1f}°C")
print(f"Actual outlet used: {actual_outlet_used:.1f}°C")
print()

# Calculate errors
heat_curve_error = abs(heat_curve_reverse - actual_outlet_used)
physics_error = abs(physics_reverse - actual_outlet_used)

print("OUTLET PREDICTION ERRORS:")
print(f"Heat Curve error: {heat_curve_error:.1f}°C")
print(f"Physics Model error: {physics_error:.1f}°C")
print()

if heat_curve_error != physics_error:
    better_model = "Heat Curve" if heat_curve_error < physics_error else "Physics Model"
    print(f"✅ SUCCESS: Models show DIFFERENT performance! {better_model} is more accurate.")
else:
    print("❌ Still identical performance - need to debug further")

print("\n🔧 This reverse prediction approach properly evaluates model accuracy!")
print("   Each model's outlet prediction is tested against what actually happened.")

## Key Benefits of Reverse Prediction Approach

1. **Different Results**: Each model uses different physics to reverse-engineer outlet temperatures
2. **Meaningful Feedback**: Adaptive learning gets real performance differences to learn from
3. **Realistic Testing**: Tests "how well does each model predict needed outlet temperature?"
4. **Proper Comparison**: Compares outlet prediction accuracy, not identical indoor temperatures

## Implementation for Full Dataset

To apply this to the full historical validation:

1. For each historical data point (State 0 → State 1):
   - **Given**: State 1 achieved indoor temperature
   - **Question**: What outlet temperature should each model have predicted in State 0?
   - **Evaluation**: Compare each model's reverse prediction against actual State 0 outlet temperature
   - **Score**: Model with smaller outlet prediction error wins

2. This gives the adaptive learning system:
   - **Real performance differences** to learn from
   - **Meaningful parameter updates** based on outlet prediction accuracy
   - **Different results** for heat curve vs physics model

The original evaluation bug was comparing both models against the same indoor temperature values, making them appear identical. This reverse approach tests each model's core competency: predicting the outlet temperature needed for desired results.